# Best Models

In [1]:
COLAB =True # IF YOU USE GOOGLE COLAB -> COLAB = True
PIP = True # IF YOU NEED INSTALL LIBRARIES -> PIP = True


In [1]:
COLAB =True # IF YOU USE GOOGLE COLAB -> COLAB = True
PIP = True # IF YOU NEED INSTALL LIBRARIES -> PIP = True

if PIP:
    !pip install transformers --upgrade
    !pip install datasets accelerate
    !pip install evaluate
    !pip install -U PyEvALL

!pip install torch
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install -U optuna

In [2]:
# Standard libraries
import os
import sys
import tempfile
import time
import ast
import json
import random

# Data manipulation
import numpy as np
import pandas as pd

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

# PEFT (Parameter-Efficient Fine-Tuning)
from peft import LoraConfig, get_peft_model, TaskType

# Evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Optuna for hyperparameter tuning
import optuna

# PyEvALL for evaluation
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory
from pyevall.reports.reports import PyEvALLReport
from pyevall.utils.utils import PyEvALLUtils


In [23]:
from pathlib import Path

if COLAB is True:
  from google.colab import drive
  drive.mount('/content/drive',force_remount=True)
  base_path = "/content/drive/MyDrive/EXISTS2025_TweetBusters"
  library_path = base_path + "/Functions"
else:
  base_path = Path.cwd().parent
  library_path = base_path / "Functions"



sys.path.insert(0, str(library_path))
from readerEXIST2025_2 import EXISTReader

Mounted at /content/drive


In [24]:
# path to the dataset, adapt this path wherever you have the dataset
dataset_path = os.path.join(base_path, "Dataset/EXIST_2025_Dataset_V0.3/")

file_train = os.path.join(dataset_path, "EXIST2025_training.json")
file_dev = os.path.join(dataset_path, "EXIST2025_dev.json")
file_test = os.path.join(dataset_path, "EXIST2025_test_clean.json")


reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test)


EnTrainTask1, EnDevTask1, EnTestTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1"), reader_test.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2, EnTestTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2"), reader_test.get(lang="EN", subtask="2")
EnTrainTask3, EnDevTask3, EnTestTask3 = reader_train.get(lang="EN", subtask="3"), reader_dev.get(lang="EN", subtask="3"), reader_test.get(lang="EN", subtask="3")


SpTrainTask1, SpDevTask1, SpTestTask1  = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1"), reader_test.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2, SpTestTask2  = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2"), reader_test.get(lang="ES", subtask="2")
SpTrainTask3, SpDevTask3, SpTestTask3  = reader_train.get(lang="ES", subtask="3"), reader_dev.get(lang="ES", subtask="3"), reader_test.get(lang="ES", subtask="3")


In [5]:
class SexismDatasetMulti(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)}
# Ejemplo de uso para task 3:

In [6]:
import os
import importlib.util
import sys
import inspect

functions_path = library_path

for filename in os.listdir(functions_path):
    if filename.endswith(".py") and not filename.startswith("__"):
        module_name = filename[:-3]
        file_path = os.path.join(functions_path, filename)

        # Cargar el módulo
        spec = importlib.util.spec_from_file_location(module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Extraer todas las funciones del módulo y cargarlas al espacio global
        for name, func in inspect.getmembers(module, inspect.isfunction):
            globals()[name] = func  # o locals()[name] si estás dentro de una función

In [7]:
def set_seed(seed=123):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)


In [8]:
best_model_name_1 = "cardiffnlp/twitter-roberta-base-2022-154m"
best_model_name_2 = "andrea-t94/roberta-fine-tuned-twitter"
best_model_name_3 = "cardiffnlp/twitter-roberta-large-2022-154m"

optimized_1 ={
    'num_train_epochs': 100,
    'learning_rate': 4.870877111004627e-05,
    'per_device_train_batch_size': 8,
    'per_device_eval_batch_size': 8,
    'logging_steps': 100,
    'r': 25,
    'lora_alpha': 40
}
optimized_2 ={
    'num_train_epochs': 100,
    'learning_rate': 9.1657096479339e-05,
    'per_device_train_batch_size': 8,
    'per_device_eval_batch_size': 8,
    'logging_steps': 100,
    'r': 8,
    'lora_alpha': 59
}
optimized_3 ={
    "num_train_epochs": 10,
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 16,
    "logging_steps": 10,
    "early_stopping_patience": 3
}

In [8]:
final_model, final_metrics = sexism_classification_pipeline_task1_LoRA(
    EnTrainTask1, EnDevTask1, EnTestTask1,
    best_model_name_1, 2, "single_label_classification",
    **optimized_1
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.614900,0.430431,0.795045,0.761155,0.775401,0.747423
2,0.406500,0.374724,0.835586,0.819753,0.786730,0.855670
3,0.396500,0.377893,0.842342,0.825871,0.798077,0.855670
4,0.380900,0.382412,0.844595,0.810959,0.865497,0.762887
5,0.300000,0.377472,0.851351,0.829016,0.833333,0.824742
6,0.292600,0.378036,0.860360,0.853081,0.789474,0.927835
7,0.365800,0.387689,0.862613,0.848635,0.818182,0.881443
8,0.277500,0.441303,0.864865,0.854369,0.807339,0.907216
9,0.274500,0.439658,0.860360,0.842640,0.830000,0.855670
10,0.211100,0.457424,0.864865,0.854369,0.807339,0.907216


Validation Results: {'eval_loss': 0.4413025677204132, 'eval_accuracy': 0.8648648648648649, 'eval_f1': 0.8543689320388349, 'eval_precision': 0.8073394495412844, 'eval_recall': 0.9072164948453608, 'eval_runtime': 3.2641, 'eval_samples_per_second': 136.024, 'eval_steps_per_second': 17.156, 'epoch': 11.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1_LoRA.csv


In [9]:
final_model_2, final_metrics_2 = sexism_classification_pipeline_task2_LoRA(
    EnTrainTask2, EnDevTask2, EnTestTask2,
    best_model_name_2, 4, "single_label_classification",
    **optimized_2
)


tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at andrea-t94/roberta-fine-tuned-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.934900,0.855416,0.679293,0.334070,0.305819,0.369941
2,0.711700,0.704722,0.739899,0.434996,0.425312,0.460572
3,0.645500,0.653015,0.755051,0.493569,0.706186,0.510388
4,0.584100,0.684533,0.744949,0.482923,0.692481,0.506073
5,0.522900,0.665524,0.752525,0.511809,0.624854,0.524330
6,0.478300,0.755824,0.737374,0.520662,0.556610,0.528648
7,0.478300,0.784191,0.737374,0.496684,0.512004,0.500556
8,0.463300,0.817132,0.757576,0.519983,0.586152,0.519165
9,0.436200,0.879674,0.760101,0.494704,0.548273,0.487299


Validation Results: {'eval_loss': 0.7558236122131348, 'eval_accuracy': 0.7373737373737373, 'eval_f1': 0.5206618090181064, 'eval_precision': 0.556609931918302, 'eval_recall': 0.52864820473644, 'eval_runtime': 1.5041, 'eval_samples_per_second': 263.272, 'eval_steps_per_second': 33.241, 'epoch': 9.0}
Prediction for TASK 2 completed. Results saved to sexism_predictions_task2_LoRA.csv


In [12]:
final_model_task3, final_metrics_task3 = sexism_classification_pipeline_task3_LoRA(
    EnTrainTask3, EnDevTask3, EnTestTask3,
    best_model_name_3, 5, "multi_label_classification",
    **optimized_3,

)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-large-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.666600,0.671050,0.455030,0.403077,0.317501,-1.093383
2,0.589000,0.599802,0.755206,0.578497,0.621059,-0.325006
3,0.573400,0.579964,0.734401,0.624144,0.668856,-0.915101
4,0.516100,0.571863,0.725468,0.713321,0.710175,0.137946
5,0.550100,0.568655,0.729208,0.684212,0.704202,-0.361977
6,0.450300,0.570128,0.737478,0.671094,0.698830,-0.491532
7,0.536200,0.571301,0.742429,0.659605,0.696682,-0.592367


2025-05-23 11:37:13,270 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:37:13,352 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
cargado 29
2025-05-23 11:39:12,732 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:39:12,791 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 11:41:12,328 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:41:12,425 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 11:43:11,983 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:43:12,049 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 11:45:11,568 - pyevall.evaluation 

2025-05-23 11:49:19,421 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:49:19,480 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.5718631148338318, 'eval_precision_macro': 0.7254678196754969, 'eval_recall_macro': 0.7133212053081498, 'eval_f1_macro': 0.7101747170818857, 'eval_ICM': 0.13794645809590753, 'eval_runtime': 8.1813, 'eval_samples_per_second': 41.314, 'eval_steps_per_second': 2.689, 'epoch': 7.0}
2025-05-23 11:50:01,490 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 11:50:01,635 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction for TASK 3 (LoRA) completed. Results saved to sexism_predictions_task3_LoRA.csv


In [29]:
best_model_name_1 = "tum-nlp/bertweet-sexism"
best_model_name_2 = "dccuchile/bert-base-spanish-wwm-uncased"
best_model_name_3 = "pysentimiento/robertuito-base-uncased".strip()


optimized_1 ={
    # Training args
    "num_train_epochs": 16,
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "logging_steps": 50,
    # LoRA args
    "rank": 8,
    "lora_alpha": 16,
    "lora_dropout": 0.1,
    "task_type": TaskType.SEQ_CLS,
    "target_modules": ["query","value"],
}
optimized_2 = {
            'output_dir': "outputs/bertin_roberta_es",
            'num_train_epochs': 16,
            'learning_rate': 5e-5,
            'per_device_train_batch_size': 16,
            'per_device_eval_batch_size': 64,
            'warmup_steps': 500,
            'lora_dropout': 0.1,
            'rank': 8,
            'lora_alpha': 16,
            'bias': "none"}

optimized_3 ={
    "num_train_epochs": 10,
    "learning_rate": 5e-5,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 16,
    "logging_steps": 10,
    "early_stopping_patience": 3
}

In [15]:
final_model, final_metrics = sexism_classification_pipeline_task1_LoRA(
    SpTrainTask1, SpDevTask1, SpTestTask1,
    best_model_name_1, 2, "single_label_classification",
    **optimized_1
)

tokenizer_config.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.576700,0.585478,0.691837,0.704501,0.720000,0.689655
2,0.540700,0.561160,0.759184,0.779851,0.760000,0.800766
3,0.485900,0.604436,0.746939,0.741667,0.812785,0.681992
4,0.425800,0.541927,0.775510,0.777328,0.824034,0.735632
5,0.455800,0.484265,0.789796,0.798434,0.816000,0.781609
6,0.399000,0.511286,0.781633,0.781186,0.837719,0.731801
7,0.397200,0.490519,0.789796,0.798434,0.816000,0.781609
8,0.391800,0.482088,0.785714,0.787018,0.836207,0.743295


Validation Results: {'eval_loss': 0.48426494002342224, 'eval_accuracy': 0.789795918367347, 'eval_f1': 0.7984344422700587, 'eval_precision': 0.816, 'eval_recall': 0.7816091954022989, 'eval_runtime': 11.6828, 'eval_samples_per_second': 41.942, 'eval_steps_per_second': 2.653, 'epoch': 8.0}
Prediction for TASK 1 completed. Results saved to sexism_predictions_task1_LoRA.csv


In [25]:
final_model_2, final_metrics_2 = sexism_classification_pipeline_task2_LoRA(
    SpTrainTask2, SpDevTask2, SpTestTask2,
    best_model_name_2, 4, "single_label_classification",
    **optimized_2
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.270100,1.222816,0.520642,0.171450,0.131062,0.247817
2,1.028300,1.137071,0.525229,0.172180,0.131307,0.250000
3,1.023800,1.057358,0.575688,0.262147,0.319469,0.300604
4,0.900200,0.909211,0.667431,0.356784,0.318624,0.408080
5,0.759100,0.846487,0.660550,0.362859,0.442144,0.405425
6,0.699000,0.825927,0.672018,0.390166,0.487912,0.425170
7,0.683200,0.813639,0.674312,0.451999,0.615823,0.456141
8,0.753200,0.809977,0.669725,0.428627,0.515321,0.442242
9,0.690300,0.811005,0.672018,0.458231,0.596579,0.455953
10,0.696100,0.808348,0.672018,0.480810,0.589548,0.470700


Validation Results: {'eval_loss': 0.8093761205673218, 'eval_accuracy': 0.676605504587156, 'eval_f1': 0.49403705040298324, 'eval_precision': 0.5968576420914551, 'eval_recall': 0.48016879692699244, 'eval_runtime': 3.2373, 'eval_samples_per_second': 134.679, 'eval_steps_per_second': 2.162, 'epoch': 16.0}
Prediction for TASK 2 completed. Results saved to sexism_predictions_task2_LoRA.csv


In [30]:
final_model_task3, final_metrics_task3 = sexism_classification_pipeline_task3_LoRA(
    SpTrainTask3, SpDevTask3, SpTestTask3,
    best_model_name_3, 6, "multi_label_classification",
    **optimized_3
)

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/858k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Precision Macro,Recall Macro,F1 Macro,Icm
1,0.591700,0.582360,0.761663,0.551513,0.559771,-0.287398
2,0.504600,0.494805,0.749585,0.713623,0.722667,0.441000
3,0.457500,0.488354,0.764724,0.700212,0.720208,0.424780
4,0.474900,0.487047,0.768015,0.696063,0.723166,0.431198
5,0.444300,0.486087,0.751608,0.726117,0.735032,0.516644
6,0.464400,0.487604,0.769739,0.703051,0.726313,0.450375
7,0.432600,0.486674,0.763683,0.713901,0.730763,0.492003
8,0.388500,0.487152,0.765281,0.702883,0.724833,0.454159


2025-05-23 12:36:55,196 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:36:55,286 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 12:37:51,939 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:37:52,026 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 12:38:49,019 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:38:49,118 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 12:39:45,852 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:39:45,931 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-05-23 12:40:42,779 - pyevall.evaluation - INFO -   

2025-05-23 12:43:37,925 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:43:38,011 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Validation Results: {'eval_loss': 0.4860874116420746, 'eval_precision_macro': 0.7516075660513671, 'eval_recall_macro': 0.7261174395653812, 'eval_f1_macro': 0.7350316802159528, 'eval_ICM': 0.5166435884831225, 'eval_runtime': 3.9077, 'eval_samples_per_second': 125.392, 'eval_steps_per_second': 7.933, 'epoch': 8.0}
2025-05-23 12:43:53,335 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM']
2025-05-23 12:43:53,593 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
Prediction for TASK 3 (LoRA) completed. Results saved to sexism_predictions_task3_LoRA.csv
